<a href="https://colab.research.google.com/github/camilavejar/COMP432-Machine-Learning-Project-Emotional-Summaries/blob/main/Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Emotional Summaries**

## **Abstract**
   Abstract here. Give an executive summary of your project: goal, methods, results, conclusions. Usually no more than 200 words.


## **Introduction**

<!-- Here you have to explain the problem that you are solving. Explain why it is important, and what are the main challenges. Mention previous attempts (add papers as references) to solve it. Mainly focus on the techniques closely related to our approach. Briefly describe your approach and explain why it is promising for solving the addressed problem. Mention the dataset and the main results achieved.

In this section, you can add **text** and **figures**. -->


Summarization is a very powerful tool, as it allows us to ingest a large amount of data in just a few sentences. However, most of the summarization tools available today are based on being factual.This means that we can only produce  objective summaries and possibly miss all the context and undertones that emotions brings in. By including emotion in the summarizing of a text, we can provide a more context-rich and complete summary to users. For example, if we use a traditional summarization model to process a news article, we will get only the facts. If we incorporate emotion detection, we can learn about any biases the reporter has, ie we are able to tell if the news article is trying to make us feel a certain way about the contents. This brings more power to the users, as they should be aware and not manipulated by what they read.


### Main challenges
The main challenges about this issue revolve around the emotion integration into the summaries, the lack of readily available datasets and the computational resources needed. There exists several summarization models like PEGASUS and T5, both transformers capable of summarizing a text input objectively.


> PEGASUS (Pre-training with Extracted Gap-sentences for Abstractive Summarization)

*   This model is transformer based and made for abstractive summaries.
*   This model is innovative in the fact that it is capable of predict any missing sentences in a summary.
*   Trained on CNN/DailyNews dataset

> T5 (Text-to-Text Transfer Transformer)

*   Thia model is transformer based and made for various tasks such as summarixation, translation, classification, etc. It is not focused on generating summaries.
*   Less computationally expensive than PEGASUS. Takes in and outputs a smaller amount of tokens.

(Other models such as BART, are also used in summarization tasks but we will be focusing on PEGASUS and T5)

Despite the existence of various models capable of producing summaries, the integration of emotion isn't as common or as widely available. It is difficult for models to understand and then reproduce emotion into words. Most of the exisitng works connecitng emotion to summaries focus on emotion detection instead of emotion integration.

As for the dataset availability, the ones available are either article-summary datasets, like the CNN/Daily Mail dataset or emotion label-text datatsets, like the GoEmotions dataset. Our summarization models needs to have something to summarize as well as the goal summary which is why the mentioned dataset is commonly used for summarization purposes. On the other hand the GoEmotions dataset has emotional texts with labels but no goal summary. This means that for this project, we will be needing a custom dataset in the format of article-emotional goal summary OR emotion labeled articles-goal summaries.

As for computational resources, since these are transformer models we cannot fully train them. This means that we can use techniques such as partially freezing the models to only fine tune the encoder layers reduce resource usage. Even with this precaution, the usage of a fast GPU is necessary to reduce training time of 1 epoch from 30 minutes to 10 minutes.

### My Approach

first thought - use a emotion classifier to label data == issue objective summaries still == inference

instead == use chat gpt as a tool
== run data processing part to get a part of the cnn daily news dataset == json files (5k limit or else gpt cant handle it) then pass it through gpt with prompt XXX

now we have article - emotional summaries == better than inference == no need for labeling

after getting data == tokenize and pass through model
freeze part of it - set up trianing argument and train

dataset to be used + results obtained

## **Methodology**
<!-- Describe the important steps you took to achieve your goal. Focus more on the most important steps (preprocessing, extra features, model aspects) that turned out to be important. Mention the original aspects of the project and state how they relate to existing work.

In this section, you can add **text** and **figures**. For instance, it is strongly suggested to add a picture of the best machine learning model that you implemented to solve your problem (and describe it).
 -->
The steps to solve this issue were:

### Preprocessing
data --gpt

### Set up the model

### Train

### Evaluate

### Fine tune and Compare


## **Experimental Setup**
<!-- Describe the datasets used for your experiments. List the machine learning techniques used to solve your problem and report the corresponding hyperparameters.

In this section, you can add **text**, **tables**, and **figures**. -->

## **Experimental Results**
<!-- Describe here the main experimental results. Critically discuss them. Compare them with results available in the literature (if applicable).

In this section, you can add **text** and **figures**, **tables**, **plots**, and code. Make sure the code is runnable and replicable. -->

results -- lots of repetiton

## **Conclusions**

<!-- Summarize what you could and could not conclude based on your experiments.
In this section, you can add **text**. -->

what could be done better -- data, more fine tuning
what coulndt be done -- test




## **References**
You can add here the citations of books, websites, or academic papers, etc.